# [데이콘 온라인 스터디] 1. 데이터 이해하기

## 칼럼을 2천개나 줄였습니다!

지난 6월부터 7월 중순까지 약 4주간에 걸쳐 원자력 상태판단대회 온라인 스터디(2기)가 진행되었습니다. 스터디에서 진행한 내용을 보다 많은 분들과 고민하고자 앞으로 3차례에 걸쳐 코드공유 게시물을 업로드할 예정입니다.

온라인 스터디와 관련한 자료는 아래 링크를 참조해 주세요.

- 데이콘 온라인 스터디 커리큘럼 [바로가기](https://www.dropbox.com/scl/fi/eaxxhf0pudm9jvckqgf4k/.papert?dl=0&rlkey=yqyrpk9eluqauoi5xjmywmp66)
- 1주차 세션 진행 자료 [바로가기](https://www.dropbox.com/scl/fi/hj22v4f47ythje8flvvm2/1.paper?dl=0&rlkey=0mrorfzb3hnvauscl459gl50z)

----

첫번째 주제는 <칼럼을 2천개나 줄였습니다!>로, 칼럼이 많은 데이터에서 불필요한 칼럼을 추출 및 삭제하는 방법에 대해 다뤄보고자 합니다.

*(이하 내용은 원자력 온라인 스터디 1, 2기에서 다루었던 내용이 포함되어 있습니다.)*

-------------

### 1. 데이터 로드

데이터 로드 시 데이콘에서 제공한 베이스라인 코드를 참조하였습니다.

다만, 상태 A에서 B로 변화하는 ~15초 구간, 즉 event_time변수 값을 기존 10에서 15로 변경하여 모든 측정치가 상태 B인 구간만을 사용하였습니다.

우승자 코드를 살펴보면 일부 팀에서 1초부터 15초 까지 1초간 변화를 주어 약 15개의 데이터셋을 활용하여 모델을 구성하기도 했는데, 각 측정치마다 변화 시점이 같다고 판단하기 어려우므로 상태 B에 대해 집중하기로 결정하였습니다.

상태 A와 B에 대해 상관관계를 도출하고자 여러 시도를 했는데, 유의미한 인사이트를 아직 얻지 못했습니다. 이 부분에 대해서는 조금 더 고민을 해봐야 할 것 같습니다.

추가로, nrows는 75로 설정하여 60초의 데이터를 가져올 수 있도록 하였습니다.

In [3]:
import multiprocessing 
from multiprocessing import Pool 
from functools import partial 
from data_loader_v2 import data_loader_v2
import os 
import pandas as pd
import numpy as np
import joblib

In [4]:
train_folder = 'train/'
train_list = os.listdir(train_folder)
train_label_path = 'train_label.csv'
train_label = pd.read_csv(train_label_path, index_col=0)

In [5]:
def data_loader_all_v2(func, files, folder='', train_label=None, event_time=15, nrows=75):   
    func_fixed = partial(func, folder=folder, train_label=train_label, event_time=event_time, nrows=nrows)  
    if __name__ == '__main__':
        pool = Pool(processes=multiprocessing.cpu_count()) 
        df_list = list(pool.imap(func_fixed, files)) 
        pool.close() 
        pool.join() 
    combined_df = pd.concat(df_list)
    return combined_df

In [6]:
# event_time=15, nrows=75 설정
train = data_loader_all_v2(data_loader_v2, train_list, folder=train_folder, train_label=train_label, event_time=15, nrows=75)

---------

### 2. 데이터 처리

본 대회의 데이터는 칼럼이 많은 것이 특징입니다. 약 5000여개의 칼럼에서 row에 따라 값이 변하지 않는 칼럼은 의미없는 칼럼으로 간주하여 해당 칼럼을 삭제하는 작업을 진행하였습니다.

1) 유니크한 값의 개수가 1인 칼럼, 2) 표준편차가 0인 칼럼을 제거하는 방식을 적용하였습니다. 두 방법이 비슷해 보이지만, 1)의 경우 1783개, 2)의 경우 1693개의 칼럼이 삭제되어 약 100개 정도 차이가 있었습니다.

이 차이에 대해서는 스터디원 '인호베이션97'님 께서 토론 게시물을 작성해 주셨습니다. [링크](https://dacon.io/competitions/official/235551/talkboard/401155?page=1&dtype=recent&ptype=pub)를 참조해주세요.

In [8]:
#원본 데이터 크기
train.shape

(49620, 5122)

In [9]:
#칼럼 drop ver1 : nunique()!=1
train_1 = train.loc[:,train.nunique()!=1]
train_1.shape

(49620, 3339)

In [44]:
#칼럼 drop ver2 : train.std()!=0
train_2 = train.loc[:,train.std()!=0]
train_2.shape

(49620, 3429)

In [11]:
X_train = train.drop(['label'], axis=1)
y_train = train['label']

X_train_1 = train_1.drop(['label'], axis=1)
y_train_1 = train_1['label']

X_train_2 = train_2.drop(['label'], axis=1)
y_train_2 = train_2['label']

단순 rf모델로 칼럼 축소 성능을 비교해보겠습니다. 사이킷런의 cross_val_score 결과상으로는 미세한 차이이나, 본 대회의 평가지표, log loss를 고려한다면 작은 차이라도 리더보드상 결과는 크게 달라질 수 있기에 2)안을 채택하여 칼럼을 drop할 수 있었습니다. (물론, train 데이터에서 drop된 칼럼은 test에서도 동일하게 drop되었습니다.)

In [42]:
#칼럼제거 성능평가
#원본 데이터세트
#랜덤 포레스트 이용해 타깃 값이 디폴트 값을 3개 교차 검증 세트로 분류 예측
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rcf=RandomForestClassifier(n_estimators=300)
scores = cross_val_score(rcf, X_train, y_train, scoring='accuracy', cv=3)
print('원본\nCV3인 경우 개별 fold 세트별 정확도 : ', scores)
print('평균 정확도 : {0:.4f}'.format(np.mean(scores)))

원본
CV3인 경우 개별 fold 세트별 정확도 :  [0.7795827 0.80259345 0.79941585]
평균 정확도 : 0.7939


In [29]:
#칼럼 drop ver1
rcf_1=RandomForestClassifier(n_estimators=300)
scores = cross_val_score(rcf_1, X_train_1, y_train_1, scoring='accuracy', cv=3)
print('CV3인 경우 개별 fold 세트별 정확도 : ', scores)
print('평균 정확도 : {0:.4f}'.format(np.mean(scores)))

CV3인 경우 개별 fold 세트별 정확도 :  [0.80912938 0.82091898 0.82025393]
평균 정확도 : 0.8168


In [46]:
#칼럼 drop ver2
rcf_2=RandomForestClassifier(n_estimators=300)
scores = cross_val_score(rcf_2, X_train_2, y_train_2, scoring='accuracy', cv=3)
print('CV3인 경우 개별 fold 세트별 정확도 : ', scores)
print('평균 정확도 : {0:.4f}'.format(np.mean(scores)))

CV3인 경우 개별 fold 세트별 정확도 :  [0.81737606 0.82128174 0.81741233]
평균 정확도 : 0.8187


결과적으로 5122개의 칼럼에서 3339여개의 칼럼으로 축소할 수 있었습니다. 

다음 게시물에서는 데이터셋의 라벨 별 분포도를 살펴보고, 균형적인 데이터 분포를 위해 고려해야 할 사항을 알아보도록 하겠습니다.

-----